In [1]:
from network import POFLNetWork, ZKFLChain
from federated_learning.client import Worker
from federated_learning.model import LeNet_Small_Quant
import federated_learning

import torch
import torch.nn as nn
import numpy as np

In [2]:
(X_train, y_train), (X_test, y_test) = federated_learning.load_mnist(1)

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100%|██████████| 9912422/9912422 [00:01<00:00, 8786495.56it/s] 


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100%|██████████| 28881/28881 [00:00<00:00, 329129.26it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100%|██████████| 1648877/1648877 [00:00<00:00, 2713526.37it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 6840405.30it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [3]:
X_train[0].shape

(60000, 1, 28, 28)

In [4]:
y_train[0]

array([5, 0, 4, ..., 5, 6, 8])

In [5]:
worker.quantized_model

LeNet_Small_Quant(
  (loss_fn): CrossEntropyLoss()
  (conv1): QuantizedConv2d(3, 6, kernel_size=(5, 5), stride=(1, 1), scale=3.4812536239624023, zero_point=72, bias=False)
  (act1): ReLU()
  (pool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): QuantizedConv2d(6, 16, kernel_size=(5, 5), stride=(1, 1), scale=1.4441901445388794, zero_point=81, bias=False)
  (act2): ReLU()
  (pool2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv3): QuantizedConv2d(16, 120, kernel_size=(4, 4), stride=(1, 1), scale=0.3627620339393616, zero_point=67, bias=False)
  (act3): ReLU()
  (linear1): QuantizedLinear(in_features=480, out_features=84, scale=0.13628940284252167, zero_point=67, qscheme=torch.per_channel_affine)
  (act4): ReLU()
  (linear2): QuantizedLinear(in_features=84, out_features=10, scale=0.0548165999352932, zero_point=58, qscheme=torch.per_channel_affine)
  (quant): Quantize(scale=tensor([2.0069]), zero_point=tensor([0]), dtype=torch.quint8)
  (dequant): DeQuantize()
)

In [21]:
model = LeNet_Small_Quant()
model.qconfig = torch.quantization.get_default_qconfig(
                'fbgemm')
print(model.qconfig)
model_prepared = torch.quantization.prepare(model)
model_int8 = torch.quantization.convert(model_prepared)

QConfig(activation=functools.partial(<class 'torch.ao.quantization.observer.HistogramObserver'>, reduce_range=True){}, weight=functools.partial(<class 'torch.ao.quantization.observer.PerChannelMinMaxObserver'>, dtype=torch.qint8, qscheme=torch.per_channel_symmetric){})


In [23]:
model.get_params()

[array([[[[-0.04413191, -0.09186491, -0.08471252,  0.0496452 ,
            0.02634946],
          [-0.04186652,  0.02070497,  0.09445369,  0.09170613,
           -0.01769409],
          [-0.06069455, -0.04501588,  0.00293596, -0.09543014,
           -0.0654113 ],
          [ 0.05122752,  0.0297306 , -0.0866945 , -0.09074506,
            0.073065  ],
          [-0.03814577, -0.09544621, -0.01050696,  0.11235705,
           -0.07674769]],
 
         [[ 0.04732366,  0.01411851, -0.08208013,  0.02968438,
            0.00406638],
          [-0.08551541, -0.08226499,  0.06058009, -0.06631197,
            0.02971233],
          [ 0.09437662, -0.04680827,  0.05399987,  0.03529698,
           -0.08346975],
          [ 0.08343302,  0.0949059 , -0.08941352, -0.06129826,
           -0.02642553],
          [ 0.08627982,  0.03573658, -0.11236435,  0.07407098,
            0.01288714]],
 
         [[-0.05191884,  0.04277205,  0.07531148, -0.0597214 ,
            0.03744031],
          [-0.01631769,  0

In [16]:
model_int8

LeNet_Small_Quant(
  (loss_fn): CrossEntropyLoss()
  (conv1): QuantizedConv2d(3, 6, kernel_size=(5, 5), stride=(1, 1), scale=1.0, zero_point=0, bias=False)
  (act1): ReLU()
  (pool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): QuantizedConv2d(6, 16, kernel_size=(5, 5), stride=(1, 1), scale=1.0, zero_point=0, bias=False)
  (act2): ReLU()
  (pool2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv3): QuantizedConv2d(16, 120, kernel_size=(4, 4), stride=(1, 1), scale=1.0, zero_point=0, bias=False)
  (act3): ReLU()
  (linear1): QuantizedLinear(in_features=480, out_features=84, scale=1.0, zero_point=0, qscheme=torch.per_channel_affine)
  (act4): ReLU()
  (linear2): QuantizedLinear(in_features=84, out_features=10, scale=1.0, zero_point=0, qscheme=torch.per_channel_affine)
  (quant): Quantize(scale=tensor([1.]), zero_point=tensor([0]), dtype=torch.quint8)
  (dequant): DeQuantize()
)